In [1]:
razor.api.libraries('fastparquet')


Name,Technology,Install Status,Created by,Created on,Modified by,Modified on
fastparquet,PYTHON,INSTALLED,Ayan Basak,"Sep 25, 2020",Ayan Basak,"Sep 25, 2020"


In [2]:
import razor


In [3]:
config = {'cpu': 8 , 'ram': 36000}

In [4]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [5]:
import razor.flow as rf
from razor.api import datasources
from razor import project_space_path
import razor
from razor import Technology, LibraryMode


In [6]:
import typing as t

# Light GBM inference

In [7]:
lib1 = razor.api.libraries(name='lightgbm').artifact()

In [8]:
@rf.block(executor=rf.ContainerExecutor(cores=config['cpu'], memory=config['ram'], gpu=1))
class Infer:
#     config: t.Any
    
    __libs__ = [lib1]
    
    def init(self):
        self.VER = 1                          # Our model version
        self.SEED = 42  
        
        #PATHS for Features
        self.ORIGINAL = project_space_path("M5Forecasting")+"/"
        self.BASE     = project_space_path('M5Forecasting/m5-simple-fe/grid_part_1.pkl')
        self.PRICE    = project_space_path('M5Forecasting/m5-simple-fe/grid_part_2.pkl')
        self.CALENDAR = project_space_path('M5Forecasting/m5-simple-fe/grid_part_3.pkl')
        self.LAGS     = project_space_path('M5Forecasting/lags_df_28.pkl')
        self.MEAN_ENC = project_space_path('M5Forecasting/mean_encoding_df.pkl')

        #LIMITS and const
        self.TARGET      = 'sales'            # Our target
        self.START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
        self.END_TRAIN   = 1913               # End day of our train set
        self.P_HORIZON   = 28                 # Prediction horizon
        self.USE_AUX     = False               # Use or not pretrained models
        self.N_CORES = psutil.cpu_count()     # Available CPU cores

        # AUX(pretrained) Models paths
        self.AUX_MODELS = './M5Forecasting/m5-aux-models/'


        #STORES ids
        self.STORES_IDS = pd.read_csv(self.ORIGINAL+'sales_train_validation.csv')['store_id']
        self.STORES_IDS = list(self.STORES_IDS.unique())


        #SPLITS for lags creation
        self.SHIFT_DAY  = 28
        self.N_LAGS     = 15
        self.LAGS_SPLIT = [col for col in range(self.SHIFT_DAY, self.SHIFT_DAY + self.N_LAGS)]
        
        self.remove_features = ['id','state_id','store_id',
                           'date','wm_yr_wk','d', self.TARGET]
        self.mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                           'enc_dept_id_mean','enc_dept_id_std',
                           'enc_item_id_mean','enc_item_id_std'] 

            
    def seed_everything(self, seed=0):
        random.seed(seed)
        np.random.seed(seed)


    ## Multiprocess Runs
    def df_parallelize_run(self, func, t_split):
        num_cores = np.min([self.N_CORES,len(t_split)])
        pool = Pool(num_cores)
        df = pd.concat(pool.map(func, t_split), axis=1)
        pool.close()
        pool.join()
        return df

    def get_data_by_store(self, store):

        # Read and contact basic feature
        df1 = pd.read_pickle(self.BASE)
#         df1 = df1[df1['store_id']==store]
        df2 = pd.read_pickle(self.PRICE).iloc[:,2:]
#         df2 = df2[df2['store_id']==store]
        df3 = pd.read_pickle(self.CALENDAR).iloc[:,2:]
        df = pd.concat([df1, df2, df3],
                        axis=1)

        # Leave only relevant store
        df = df[df['store_id']==store]

        # With memory limits we have to read 
        # lags and mean encoding features
        # separately and drop items that we don't need.
        # As our Features Grids are aligned 
        # we can use index to keep only necessary rows
        # Alignment is good for us as concat uses less memory than merge.
        df2 = pd.read_pickle(self.MEAN_ENC)[self.mean_features]
        df2 = df2[df2.index.isin(df.index)]

        df = pd.concat([df, df2], axis=1)
        del df2 # to not reach memory limit 
        gc.collect()

        df3 = pd.read_pickle(self.LAGS).iloc[:,3:]
        df3 = df3[df3.index.isin(df.index)]


        df = pd.concat([df, df3], axis=1)
        del df3 # to not reach memory limit 
        gc.collect()

        # Create features list
        features = [col for col in list(df) if col not in self.remove_features]
        features = [col for col in features if col not in ['Unnamed: 0']]
        
        del df # to not reach memory limit 
        gc.collect()
        
        return features

    # Recombine Test set after training
    def get_base_test(self):
        base_test = pd.DataFrame()

        for store_id in self.STORES_IDS:
            temp_df = pd.read_pickle(project_space_path('M5Forecasting/test_'+store_id+'.pkl'))
            temp_df['store_id'] = store_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)

        return base_test


    ########################### Helper to make dynamic rolling lags
    #################################################################################
    def make_lag(self, LAG_DAY):
        lag_df = base_test[['id','d',self.TARGET]]
        col_name = f'sales_lag_{str(LAG_DAY)}'
        lag_df[col_name] = lag_df.groupby(['id'])[self.TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
        return lag_df[[col_name]]


    def make_lag_roll(self, LAG_DAY, base_test):
        shift_day = LAG_DAY[0]
        roll_wind = LAG_DAY[1]
        lag_df = base_test[['id','d',self.TARGET]]
        col_name = f'rolling_mean_tmp_{str(shift_day)}_{str(roll_wind)}'
        lag_df[col_name] = lag_df.groupby(['id'])[self.TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
        return lag_df[[col_name]]
    
    def run(self):
        ########################### Predict
        #################################################################################

        # Create Dummy DataFrame to store predictions
        all_preds = pd.DataFrame()

        # Join back the Test dataset with 
        # a small part of the training data 
        # to make recursive features
        base_test = self.get_base_test()

        # Timer to measure predictions time 
        main_time = time.time()

        ROLS_SPLIT = []

        for i in [1,7,14]:
            for j in [7,14,30,60]:
                ROLS_SPLIT.append([i,j])

        # Loop over each prediction day
        # As rolling lags are the most timeconsuming
        # we will calculate it for whole day
#         for PREDICT_DAY in range(1,29):    
#             print('Predict | Day:', PREDICT_DAY)
#             start_time = time.time()

        # Make temporary grid to calculate rolling lags
        grid_df = base_test.copy()
        lag_param_df = pd.DataFrame()
#             grid_df = pd.concat([grid_df, self.df_parallelize_run(self.make_lag_roll, ROLS_SPLIT)], axis=1)
        for lag_param in ROLS_SPLIT:
            lag_param_df = pd.concat([lag_param_df, self.make_lag_roll(lag_param,base_test)], axis=1)
        grid_df = pd.concat([grid_df, lag_param_df], axis=1)
            
        for store_id in self.STORES_IDS:
            self.logger.info(store_id)

            # Read all our models and make predictions
            # for each day/store pairs
            model_path = project_space_path('M5Forecasting/lgb_model_'+store_id+'_v'+str(self.VER)+'.bin') 
            if self.USE_AUX:
                model_path = self.AUX_MODELS + model_path

            estimator = pickle.load(open(model_path, 'rb'))

            for PREDICT_DAY in range(1,29):    
            
                day_mask = base_test['d']==(self.END_TRAIN+ PREDICT_DAY)
                store_mask = base_test['store_id']==store_id

                mask = (day_mask)&(store_mask)
                MODEL_FEATURES = self.get_data_by_store(store_id)
                base_test[self.TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

            # Make good column naming and add 
            # to all_preds DataFrame
            temp_df = base_test[day_mask][['id',self.TARGET]]
            temp_df.columns = ['id','F'+str(PREDICT_DAY)]
            if 'id' in list(all_preds):
                all_preds = all_preds.merge(temp_df, on=['id'], how='left')
            else:
                all_preds = temp_df.copy()

            print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                          ' %0.2f min total |' % ((time.time() - main_time) / 60),
                          ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
            del temp_df

        all_preds = all_preds.reset_index(drop=True)

        submission = pd.read_csv(self.ORIGINAL+'sample_submission_accuracy.csv')[['id']]
        submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
        submission.to_csv(self.ORIGINAL+'submission_v'+str(self.VER)+'.csv', index=False)
    

In [9]:
infer = Infer()

In [10]:
p = rf.Pipeline("Walmart Inference Pipeline_gpu", targets=[infer])

In [11]:
deployed_pipeline = razor.api.engines('DS-Engine-Load-Test').execute(pipeline=p)


In [12]:
deployed_pipeline

PlatformPipelineRun(project_id='c0b26d85-a4c9-44f5-9d0e-f540dd2de644', pipeline_id='26ea9982-091e-11eb-bf49-0242ac110006', pipeline_name='Walmart Inference Pipeline_gpu', pipeline_run_id='63ee22b9-94a4-4e08-8424-8bbf00b1301f', pipeline_version=None, comment=None, created_on='2020-10-08T04:24:27.718+00:00', start_time=None, end_time=None, eta=0, run_at=1602131067718, run_duration=0, compute_time=0, wait_time=67, ran_by_user=RanByUser(user_name='souryadipta das', email='souryadipta.das@razorthink.com'), status='IN_PROGRESS', block_status=[PlatformPipelineBlockRun(pipeline_run_id='63ee22b9-94a4-4e08-8424-8bbf00b1301f', pipeline_name='Walmart Inference Pipeline_gpu', pipeline_status='IN_PROGRESS', block_id='eea23515-c76f-44ca-9b23-e7dd731dd1e0', block_run_id='10c996ec-6aea-4e95-8995-f7b84764d440', block_name='Infer_1', resource_spec=ResourceAllocated(cores=8, memory=36000, use_gpu=True, gpu=1, use_gpu_if_available=False, cluster_configuration={'worker': None, 'workerCore': None, 'workerMemory': None, 'numberOfInstances': None, 'maxServerCount': None}, additional_env={}, run_env='PYTHON'), technology='PYTHON', status='READY_TO_GO', containers=[], cluster_id='eea23515-c76f-44ca-9b23-e7dd731dd1e0', _log=None, _metric=None, wait_time=0, compute_time=0, start_time=None, end_time=None, queued_at='2020-10-08T04:24:28.000+00:00')], run_number=346, pipeline_variable_list=[], block_run_details=[BlockRunDetail(block_id='eea23515-c76f-44ca-9b23-e7dd731dd1e0', block_run_id='10c996ec-6aea-4e95-8995-f7b84764d440', block_name='Infer_1', technology='PYTHON', log_path=None, input_parameters='{}', output_parameters=None, status='READY_TO_GO', created_at='2020-10-08T04:24:27.824+00:00', queued_at='2020-10-08T04:24:28.000+00:00', start_time=None, end_time=None, compute_time=0, wait_time=0, run_time=0, block_run_output_list=[], resource_allocated=ResourceAllocated(cores=8, memory=36000, use_gpu=True, gpu=1, use_gpu_if_available=False, cluster_configuration={'worker': None, 'workerCore': None, 'workerMemory': None, 'numberOfInstances': None, 'maxServerCount': None}, additional_env={}, run_env='PYTHON'))], pipeline_json='{\n  "id": "26ea9982-091e-11eb-bf49-0242ac110006",\n  "projectId": "c0b26d85-a4c9-44f5-9d0e-f540dd2de644",\n  "name": "Walmart Inference Pipeline_gpu",\n  "description": "",\n  "blocks": [\n    {\n      "name": "Infer_1",\n      "id": "eea23515-c76f-44ca-9b23-e7dd731dd1e0",\n      "artifact_id": null,\n      "class_name": "Infer",\n      "code": "eJyVWetvG9l1nxfJIUW9ZUl+aZXtZuPdrmRKlmQ5MZzKlLyr0KYFSY7jJMZ0OHfIS4rk0JyhLWXJTTZev4LrdotMgKBBAakognzptv3Ub/ulf8P+EUXR7/1QoP2dO5RE+bFoRfDy3nPP3Dn3vM/Rr4zfuWcV+ScmnKrXYo2ys1N1Z3vmoThtNV2+x5p24Fr+jlt1A69uOVXb98Pfhhe4Isasmr3zpj1hFlrlalCu+6Ewgr2GC6CIrdeLbjPk2gVuXOAJbvKYiHuFiusE2H4SdkKRsCzmOZYV5n0xPZ/J2JmF+cUl+4q9MLe4bC9llhaLhYWiO3dlebnIwnwQbuKpCyJlWTWPtaounhQmFna53p1WywUf0/sSL4lV027urbNQvDe/wDI4bn5m+crc0swCm1+cWZ6bvzwzf3nJXbzMMgVnGWhD3Ud+7Db9sleXTy5ddlzbySzOLMwvsZkF54o9Y2eWrsw4bmHu0mLGmVuavxS2bGGU6+UgBJ8GrGK5WrWKrboT4JAu9waPuSe3AAa03+qyzjrkm5n1mLtNiye474WcmlPwTeSy2f9ialuxFKZhVAON6Y9VZkwoWOlYxR7TroFZXM5imCXkLI6ZKWcJzJJyZrIUxiTrw5hiaYx9rB9jmg1g7A8G9gcP6OQhNohxOBjZH20rFZ0NTSgHKhs+TXtjwSnAxvbHDxR55pg8YYKNYJxko2zsCyU4DYyJ7ndyQnmsrUa4Z9gpNs6G2QSbZISTLCUAPcvOsLPsHDvPptg7pTgg55iypYQX8mDBh6L/1uINr+k6tg+GlUKhXgzFhydgF2uLM3651qi6M0X3YqlZZlbDbgbW3Gxjp/p/RZ7//yBfipDPnUSu2iXfYkVrfjnafveVs1y7brl1x2NYAS9Civl21fXDnHJrL5E790JMzV58jQS7tTsD5XerPu5+Rj5gBU0ygId2tcxsUrhZx38IPfIDPGqVWZgb5O9FeqTl9NxWdqgE6d03O2pb/RvtgQH+fvCB0GYzodAdrxo+DcXU1XJjL+BefaZcb7SCmeUZ1y5eWZwvFpcLzuVroUhdrZb9wPFqjWth7ntZLU5nkH2KKWn2s2QLs1ernmNX/WuzPdhCA0VEHLkZmhogGt7HfFSz9prWox2IFdBBcMdy7AAoFnErFAM9ED8gWyUAcxs9OIO9oGOkcuDWXkE6BBFSAF+h/3gNxBtba2uroRhpND3yU5bfsB0Xcg44CLy9uf7xen7lJtCur2ytQV4bm+vZNbLYlZtr+dUVOuDmysdbgNxaW8lba/lsKOLbK5sfr22Hom8Ls21re3NlPR+KJB44mm9Yn+Dwn97GPHFna81aufMTPNjwW0EZapF0Gi3L8Vr1ANt5K3t7cw2vSAHJunV7de0mFlqDeNp0bWZJ2ae2tgnLWl/FpkHMx3mtevlBC5xObn2yfmPbWl25B2DeiiiONe16CZspWlpbGzfXQfJg0615D12r6NpBqwnVFP1Sc4/WONbw3WoRroov8rO5RHZeUeNqXB9Q

In [13]:
razor.api.engines('DS-engine').health()

PlatformEngineHealthList(slave_usage_array=[PlatformEngineHealth(server_ip='172.16.104.139', server_type='INFRASTRUCTURE', server_usage_info=[ServerUsageInfo(label='Current cpu usage', graph_type='bar', unit='', total=16, used=3, available=13, value='CORE'), ServerUsageInfo(label='Current memory usage', graph_type='bar', unit='GB', total=40802189312.0, used=3221225472.0, available=37580963840.0, value='RAM'), ServerUsageInfo(label='Current gpu usage', graph_type='bar', unit='', total=0, used=0, available=0, value='GPU')]), PlatformEngineHealth(server_ip='172.16.114.99', server_type='TECHNOLOGY', server_usage_info=[ServerUsageInfo(label='Current cpu usage', graph_type='bar', unit='', total=32, used=13, available=19, value='CORE'), ServerUsageInfo(label='Current memory usage', graph_type='bar', unit='GB', total=123480309760.0, used=91704262656.0, available=31776047104.0, value='RAM'), ServerUsageInfo(label='Current gpu usage', graph_type='bar', unit='', total=0, used=0, available=0, value='GPU')])])

In [ ]:
razor.api.engines('DS-engine').queue()

In [15]:
lib2 = razor.api.libraries(name='pathos')